# Electoral district

In [1]:
import json
import pandas as pd
from faker import Faker

In [2]:
post_code_df = pd.read_csv('_62_postcode.csv').drop_duplicates(subset=['จังหวัด','อำเภอ'])
electoral_district_df = pd.read_csv('66_Vote62_election-zones.csv')

In [3]:
electoral_district_df

,จังหวัด,อำเภอ,ตำบล,เขตเลือกตั้ง,index,เทศบาล,เขตเลือกตั้ง.1
0,กระบี่,เกาะลันตา,เกาะกลาง,3,กระบี่เกาะลันตาเกาะกลาง,NaN,3
1,กระบี่,เกาะลันตา,เกาะลันตาน้อย,3,กระบี่เกาะลันตาเกาะลันตาน้อย,NaN,3
2,กระบี่,เกาะลันตา,เกาะลันตาใหญ่,3,กระบี่เกาะลันตาเกาะลันตาใหญ่,NaN,3
3,กระบี่,เกาะลันตา,คลองยาง,3,กระบี่เกาะลันตาคลองยาง,NaN,3
4,กระบี่,เกาะลันตา,ศาลาด่าน,3,กระบี่เกาะลันตาศาลาด่าน,NaN,3
...,...,...,...,...,...,...,...
7482,อุบลราชธานี,สิรินธร,ฝางคำ,7,อุบลราชธานีสิรินธรฝางคำ,NaN,7
7483,อุบลราชธานี,เหล่าเสือโก้ก,แพงใหญ่,3,อุบลราชธานีเหล่าเสือโก้กแพงใหญ่,NaN,3
7484,อุบลราชธานี,เหล่าเสือโก้ก,โพนเมือง,3,อุบลราชธานีเหล่าเสือโก้กโพนเมือง,NaN,3
7485,อุบลราชธานี,เหล่าเสือโก้ก,หนองบก,3,อุบลราชธานีเหล่าเสือโก้กหนองบก,NaN,3


In [4]:
district_list = list()
electdistrict = dict()
for g, g_df in electoral_district_df.groupby(['จังหวัด', 'อำเภอ']):
    province, district = g
    electoral_nums = g_df['เขตเลือกตั้ง.1'].drop_duplicates().astype(int)
    districts = g_df['อำเภอ'].drop_duplicates().values

    dks = [f'{province}-{el_n}' for el_n in electoral_nums]
    
    for dk, en in zip(dks, electoral_nums):
        d = electdistrict.get(dk, dict(district_list=list(), province=province, electoralDistrictNumber=en))
        d['district_list'] += districts.tolist()
        electdistrict[dk] = d
    for d in districts:
        district_list.append(
            dict(district=d, province=province, electoral=[dict(fk=fk) for fk in dks]))

In [5]:
with open('district_province_list.json', 'w') as fp:
    json.dump(district_list, fp, ensure_ascii=False)

In [6]:
with open('electoral_district_table.json', 'w') as fp:
    json.dump(electdistrict, fp, ensure_ascii=False)

# Candidates data

In [66]:
import re
import requests
import os
import numpy as np

In [8]:
electoral_dir = 'electorals'
os.makedirs(electoral_dir, exist_ok=True)

In [9]:
candidates_df = pd.read_csv('66_WV_Candidates.csv')
party_number_df = pd.read_csv('66_WV_Party.csv')
pm_candidates_df = pd.read_csv('66_WV_PMCandidate.csv')
twfu_parties=pd.read_csv('./Parties_exported_1.csv').set_index('Id',)
twfu_people=pd.read_csv('./People_exported_1.csv')

In [10]:
candidates_df.loc[candidates_df.zone.isna(), 'zone'] = -1
# change data type
candidates_df.loc[:,'zone'] = candidates_df.zone.astype(int)

In [11]:
# set party with no number to -1
party_number_df.loc[:, 'partylist_no'] = party_number_df.partylist_no.apply(
    lambda x: int(x) if str(x).isdigit() else -1)

In [12]:
# filter
twfu_parties = twfu_parties[twfu_parties.PartyType == 'พรรค']
# get url
def get_url(s):
    if isinstance(s, str):
        url = re.sub('.*\((.*)\)','\\1', s)
        if url != 'undefined': return url
    return ''
twfu_parties.loc[:,'image_url'] = twfu_parties.Images.apply(get_url)

In [13]:
# create table
party_data = twfu_parties[['image_url', 'Name']].set_index('Name').to_dict()

In [14]:
# add number
party_data['number'] = party_number_df.set_index('party').to_dict()['partylist_no']

In [15]:
# add website
party_data['website'] = twfu_parties[~twfu_parties['Website'].isna()][
    ['Website', 'Name']].set_index('Name').to_dict()

In [16]:
people_image_url = twfu_people.set_index('Name').Images.apply(get_url).to_dict()

In [17]:
# add Prime minister candidates
party_data['pm_candidates'] = dict()
for party, party_df in pm_candidates_df.sort_values('order').groupby('party'):
    party_data['pm_candidates'][party] = [{'Name': name,
                                           'Image': people_image_url.get(name,'')}
                                          for name in party_df.name]

In [18]:
# promisetracker party
resp = requests.get('https://raw.githubusercontent.com/wevisdemo/promise-tracker/main/data/parties.json')
promisetracker_parties = resp.json()

In [19]:
party_data['promisetracker_urls'] = dict()
for party in promisetracker_parties:
    party_name_mabe_many = party['name']
    for party_name in party_name_mabe_many.split('/'): # อนาคตใหม่/ก้าวไกล
        party_data['promisetracker_urls'][party_name] =\
            'https://promisetracker.wevis.info/explore?party='+party_name_mabe_many

In [20]:
# law watch
resp = requests.get('https://raw.githubusercontent.com/wevisdemo/law-watch/main/src/data/parties.ts')
law_watch_parties_string = re.findall('\[[^\]]*\]', resp.content.decode().replace('\n',' '))

In [21]:
party_data['law_watch_urls'] = dict()
for list_string in law_watch_parties_string[:2]:
    party_list = json.loads(re.sub('\'', '"',list_string))
    for party in party_list:
        party_data['law_watch_urls'][party] = 'https://wevis.info/law-watch#investigate-section'

In [22]:
# They work for us
# if the party has ever been a government or opposition
theyworkforus_parties = twfu_parties[~twfu_parties.PartyGroup.isna()].Name.tolist()
party_data['theyworkforus_urls'] = {party: f'https://theyworkforus.wevis.info/party/{party}'
                                for party in theyworkforus_parties}

In [23]:
people_party_history_ep = (
    'https://sheets.wevis.info/api/v1/db/public/shared-view/'
    '572c5e5c-a3d8-440f-9a70-3c4c773543ec'
    '/rows'
    '?'
    'nested[PeoplePartyHistory][fields]=Party,EstablishedDate'
    '&'
    'f=Name,PeoplePartyHistory,Id'
    '&'
    'l=900')
resp = requests.get(people_party_history_ep)
people_history_table = resp.json()['data']['list']

In [24]:
DISSOVED_DATE = '2023-03-01'

In [41]:
person_is_government = dict()
person_is_opposition = dict()
for person in people_history_table:
    for party in person['PeoplePartyHistory']:
        if party['EstablishedDate'] is not None and (party['EstablishedDate'] > DISSOVED_DATE):
            person['PeoplePartyHistory'].remove(party)
        else:
            party_id = party['Party']['Id']
            if party_id in twfu_parties.index:
                party_group = twfu_parties.loc[party_id].PartyGroup
                person_is_government[person['Name']] =\
                    person_is_government.get(person['Name'], False) or (party_group == 'ร่วมรัฐบาล')
                person_is_opposition[person['Name']] =\
                    person_is_opposition.get(person['Name'], False) or (party_group == 'ฝ่ายค้าน')

In [44]:
party_data['is_government'] = (twfu_parties[~twfu_parties['PartyGroup'].isna()]\
                                .set_index('Name')['PartyGroup'] == 'ร่วมรัฐบาล')\
                                .to_dict()
party_data['is_opposition'] = (twfu_parties[~twfu_parties['PartyGroup'].isna()]
                               .set_index('Name')['PartyGroup'] == 'ฝ่ายค้าน')\
                                .to_dict()

In [81]:
twfu_people.set_index('Name')[['Birthdate','Education','ExOccupation0']].loc['ประยุทธ์ จันทร์โอชา','Birthdate']

'1954-03-21'

In [90]:
person_table_name_as_index = twfu_people.set_index('Name')

In [91]:
def data_getter(key, person_name):
    try:
        return person_table_name_as_index[~person_table_name_as_index[key].isna()].loc[person_name, key]
    except KeyError:
        return None

In [92]:
people_list = []
party_table = dict()
for i, row in candidates_df.iterrows():
    if row.party not in party_table.keys():
        party_table[row.party] = dict(
            Name=row.party,
            Number=party_data['number'][row.party],
            Image=party_data['image_url'].get(row.party),
            Candidate=party_data['pm_candidates'].get(row.party, []),
            PastGovernment=party_data['is_government'].get(row.party, False),
            PastOpposition=party_data['is_opposition'].get(row.party, False),
            # TODO: fix this on production
            Policy=f'https://staging.election66.wevis.info/policyshop/party/{row.party}',
            Promise=party_data['promisetracker_urls'].get(row.party),
            Law=party_data['law_watch_urls'].get(row.party),
            Others=party_data['theyworkforus_urls'].get(row.party),
            Website=party_data['website'].get(row.party),
            PartyList=list()
        )
    is_past_mp = person_is_government.get(row['name'], False) or person_is_opposition.get(row['name'], False)
    if row.mptype == 'เขต':
        people_list.append(dict(
            Name=row['name'],
            Number=row['number'],
            Birthdate=data_getter('Birthdate', row['name']),
            Education=data_getter('Education', row['name']),
            ExOccupation=data_getter('ExOccupation0', row['name']),
            Party=row.party,
            Province=row.province,
            Zone=row.zone,
            Image=people_image_url.get(row['name']),
            PastMP=is_past_mp,
            PastGovernment=person_is_government.get(row['name'], False),
            PastOpposition=person_is_opposition.get(row['name'], False),
        ))
    elif row.mptype == 'บัญชีรายชื่อ':
        party_table[row.party]['PartyList'].append(dict(
            Name=row['name'],
            Number=row.number,
            PastMP=is_past_mp,
            # PastGovernment=person_is_government.get(row['name'], False),
            # PastOpposition=person_is_opposition.get(row['name'], False),
        ))

In [93]:
for dist_key, district in electdistrict.items():
    electoral_people_list = [p for p in people_list
                             if p['Province'] == district['province'] # filter out
                             and p['Zone'] == district['electoralDistrictNumber']]
    
    electoral_parties = [p['Party'] for p in electoral_people_list if p['Number'] != -1]
    electoral_parties.sort(key=lambda p: party_table[p]['Number'])
    file_path = os.path.join(electoral_dir,f'{dist_key}.json')
    electoral_data = dict(People=electoral_people_list,
                          Parties=electoral_parties)
    
    with open(file_path,'w') as fp:
        json.dump(electoral_data, fp, ensure_ascii=False)

In [94]:
with open('parties.json', 'w') as fp:
    json.dump(party_table, fp, ensure_ascii=False)